In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pylab 
import scipy.stats as stats
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score

from sklearn.pipeline import Pipeline


from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.svm import LinearSVC

from sklearn.feature_selection import RFE

import collections

import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second

In [ ]:
cd C:\Users\Comp_Imaging_1\Desktop\Thesis

In [ ]:
data = pd.read_excel('SPECT.xlsx', sheet_name = 'Sheet1')
data.shape

In [ ]:
# In practice, feature selection should be done after data pre-processing,
# so ideally, all the categorical variables are encoded into numbers,
# and then you can assess how deterministic they are of the target

# here for simplicity I will use only numerical variables
# select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

In [ ]:
#Dropping NA
data = data.dropna(axis = 1, thresh = round((0.5)*data.shape[0])) #removing features existing in less than half of the samples 
data = data.dropna(axis = 0, thresh = round((0.5)*data.shape[1])) #removing samples having less than half of the features 
data = data.fillna(0) #filling missing values with zeroes
print(data.shape)

In [ ]:
#def correlation(dataset, threshold):
#    col_corr = set()  # Set of all the names of correlated columns
#    corr_matrix = dataset.corr()
#    for i in range(len(corr_matrix.columns)):
#        for j in range(i):
#            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
#                colname = corr_matrix.columns[i]  # getting the name of column
#                col_corr.add(colname)
#    return col_corr

In [ ]:
#Change number of selected features
number_of_selected_features = 3

In [ ]:
#Feature Selection Algorithms
fs1 = RFE(LinearSVC(), n_features_to_select=number_of_selected_features, verbose=2)
fs2 = RFE(RandomForestClassifier(n_estimators=100), n_features_to_select=number_of_selected_features, verbose=2)

In [ ]:
#Pipelines

#pipeline4
pipe4 = Pipeline([('scaler', StandardScaler()),
                 ('fs', fs2)])

gs = pipe4

In [ ]:
# Checking training set model performance for detecting underfitting
start = time.time()
gs = gs.fit(np.array(data.drop(labels=['grup'], axis=1)), data['grup'])

training_pred = gs.predict(np.array(data.drop(labels=['grup'], axis=1)))
end = time.time()
print(confusion_matrix(data['grup'],training_pred))
print('Computation Time:',end - start)
winsound.Beep(frequency, duration)

In [ ]:
Estimated_Time = ((end - start)*data.shape[0])
print('Estimated Time:',Estimated_Time/60,'min','or',Estimated_Time/3600,'h')

In [ ]:
start = time.time()
print('Program start to run at',time.localtime())
pred=[]
response=[]
selectedfeatures=[]

for i in range(data.shape[0]):
    
    test = data.iloc[[i]] 
    train = data.drop(data.index[i])
    
 #   corr_features = correlation(train.drop(labels=['grup'], axis=1), 0.9)
 #   print('correlated features: ', len(set(corr_features)))
    
    
    # run gridearch
    gs = gs.fit(np.array(train.drop(labels=['grup'], axis=1)), train['grup'])
    pred.append(int(gs.predict(np.array(test.drop(labels=['grup'], axis=1)))))
    
    # storing true labels
    response.append(int(test['grup']))
    # storing the selected features in a list
    selectedfeatures.append(tuple(gs.steps[1][1].get_support(indices=True)))          #for RF-RFE
    
print(pred)
print(response)
 

counter=collections.Counter(selectedfeatures)

print('Most Common Selected Features:',np.array(counter.most_common))    #change according to number of selected features

end = time.time()
print('Computation Time:',(end - start)/60,'min')
winsound.Beep(frequency, duration)

In [ ]:
counter.most_common

In [ ]:
#Check Most Common Selected Features list above. As an example, (0,1,2):48 means that features having indices 0,1,2
#are selected 48 times out of 48+26=74 trials. Names of these features can be learned by writing index values below in 
#df = data.drop(labels=['grup'], axis=1).iloc[:,[0, 1, 2 <---write indices as here]]. 


df = data.drop(labels=['grup'], axis=1).iloc[:,[0, 1, 2]]   #change indices 
df.columns